### Dataset & DataLoader 살펴보기
- Pytorch에서 배치크기만 데이터를 조절하기 위한 메카니즘
- Dataset : 사용 데이터를 기반으로 사용자 정의 클래스 작성
- DataLoader : 지정된 Dataset에서 지정된 batch size만큼 피처와 타겟을 추출하여 전달

[1] 모듈 로딩 및 데이터 준비 <hr>

In [2]:
# 모듈 로딩
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

In [3]:
# 데이터 준비
# 임의의 데이터 생성
x_data = torch.IntTensor([[10, 20, 30], [20, 30, 40], [30, 40, 50], [40, 50, 60], [50, 60, 70]]) # 5 X 3
y_data = torch.FloatTensor([[20], [30], [40], [50], [60]]) # 5 X 1

In [4]:

print(f'x_data => {x_data.shape} {x_data.ndim}D')
print(f'y_data => {y_data.shape} {y_data.ndim}D')

x_data => torch.Size([5, 3]) 2D
y_data => torch.Size([5, 1]) 2D


[2] 데이터셋 생성 <hr>

[2-1] TensorDataset 활용 : Dataset의 sub_class

In [5]:
# TensorDataset 클래스 로딩
from torch.utils.data import TensorDataset

In [6]:
dataset = TensorDataset(x_data, y_data)
dataset

In [7]:
dataset.tensors # tensor 정보 출력

(tensor([[10, 20, 30],
         [20, 30, 40],
         [30, 40, 50],
         [40, 50, 60],
         [50, 60, 70]], dtype=torch.int32),
 tensor([[20.],
         [30.],
         [40.],
         [50.],
         [60.]]))

In [8]:
#__getitime__() 메서드 호출
dataset[0]

(tensor([10, 20, 30], dtype=torch.int32), tensor([20.]))

In [9]:
len(dataset)

5

[2-2] 사용자 정의 데이터셋 생성

In [10]:
# 데이터 준비 
filename = '../data/iris.csv'

In [11]:
irisDF = pd.read_csv(filename)

In [12]:
irisNP = np.loadtxt(filename, delimiter=',', usecols=[0, 1, 2, 3], skiprows=1)
irisNP[:2]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

In [13]:
# 데이터 타입 체크
type(irisDF), type(irisNP), irisDF.__class__, irisNP.__class__.__name__

(pandas.core.frame.DataFrame,
 numpy.ndarray,
 pandas.core.frame.DataFrame,
 'ndarray')

In [14]:
if irisDF.__class__.__name__ == 'DataFrame' :
    print('DF')
else : 
    print('---')

DF


In [15]:
isinstance(irisDF, pd.DataFrame), isinstance(irisNP, pd.DataFrame), isinstance(irisNP, np.ndarray)

(True, False, True)

In [16]:
isinstance([10], list), isinstance({'A':22}, list) 

(True, False)

In [83]:
# 사용자 정의 DataSet 클래스
class DLDataset(Dataset) :
     
    #초기화 함수 콜백 함수
    def __init__(self, x_data, y_data) :
        super().__init__()
        
        # x, y 데이터 --> ndarray
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data
        
        # x, y 데이터 --> tensor
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.FloatTensor(y_data)
        print(self.target.shape, self.target.ndim)


    # 데이터셋의 개수 확인 함수 콜백 함수
    def __len__(self) :
        return self.target.shape[0]

    # 특정 인덱스 데이터+라벨 반환 콜백 함수 (Call back function)
    def __getitem__(self, index):
        return self.feature[index], self.target[index]

In [84]:
# 피처와 라벨로 분리
featureDF = irisDF[irisDF.columns[:-1]]
# targetSR = irisDF[irisDF.columns[-1]].to_frame()
targetSR = irisDF[irisDF.columns[-1]]

print(f'featureDF => {featureDF.shape}, {featureDF.ndim}D')
print(f'targetSR => {targetSR.shape}, {targetSR.ndim}D')

featureDF => (150, 4), 2D
targetSR => (150,), 1D


In [85]:
# object 타입 타겟 => int 타입 타겟 변환
from sklearn.preprocessing import LabelEncoder

targetNP = LabelEncoder().fit_transform(targetSR) # 다시 쓸 일 없을 때는 이렇게 써도 됨.
targetNP = targetNP.reshape(-1, 1)

print(f'targetNP => {targetNP.shape}, {targetNP.ndim}')

targetNP => (150, 1), 2


In [1]:
# 데이터셋 생성
my_dataset2 = DLDataset(featureDF, targetNP)

NameError: name 'DLDataset' is not defined

In [21]:
my_dataset2[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000], dtype=torch.float64),
 tensor([0], dtype=torch.int32))

[2-3] 학습용, 검증용, 테스트용 Dataset <hr> 

In [22]:
# 파이토치
from torch.utils.data import random_split

# 학습용, 검증용, 테스트용 데이터 비율
seed = torch.Generator().manual_seed(42)
trainDS, validDS, testDS = random_split(my_dataset2, [0.7, 0.1, 0.2], generator = seed)

print(f'trainDS => {len(trainDS)}개\t validDS => {len(validDS)}개\t testDS => {len(testDS)}개\n')

print(f'[Subset 속성]\nindices : {trainDS.indices}\ndataset : {trainDS.dataset}\n')
print(f'[Subset 속성]\nindices : {validDS.indices}\ndataset : {validDS.dataset}')

trainDS => 105개	 validDS => 15개	 testDS => 30개

[Subset 속성]
indices : [42, 95, 30, 64, 52, 35, 130, 40, 82, 17, 108, 94, 68, 97, 117, 127, 41, 44, 57, 140, 149, 32, 23, 102, 16, 113, 71, 18, 67, 66, 0, 25, 101, 112, 91, 3, 59, 116, 86, 84, 106, 142, 43, 39, 26, 98, 93, 20, 87, 19, 120, 114, 7, 63, 76, 89, 36, 45, 37, 56, 58, 122, 51, 145, 24, 21, 105, 62, 15, 11, 48, 133, 88, 50, 6, 134, 111, 8, 49, 75, 69, 124, 4, 147, 80, 100, 99, 141, 47, 107, 13, 109, 129, 28, 38, 53, 121, 5, 55, 31, 73, 74, 54, 29, 12]
dataset : <__main__.DLDataset object at 0x000001F32C32A970>

[Subset 속성]
indices : [22, 104, 81, 1, 103, 125, 85, 2, 96, 128, 27, 118, 77, 110, 146]
dataset : <__main__.DLDataset object at 0x000001F32C32A970>


[3] DataLoader 생성 : 학습용, 검증용, 테스트용 <hr>

In [23]:
# DataLoader 생성
# drop_last 매개변수 : 배치 사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 [기본 : False]
batch = 5
trainDL =DataLoader(trainDS, batch_size=batch) # batch_size : 구간을 몇개로 나눌 건지.
validDL =DataLoader(validDS, batch_size=batch) #drop_last : 배치
testDL =DataLoader(testDS, batch_size=batch)  

In [24]:
# Epoch 당 반복 단위
print(f'batch_size : {batch}\n')
print(f'trainDS => {len(trainDS)}개\t validDS => {len(validDS)}개\t testDS => {len(testDS)}개\n')
print(f'trainDL => {len(trainDL)}개\t validDS => {len(validDL)}개\t testDS => {len(testDL)}개\n')

batch_size : 5

trainDS => 105개	 validDS => 15개	 testDS => 30개

trainDL => 21개	 validDS => 3개	 testDS => 6개



In [25]:
# DataLoader 속성

for _, (feature, target) in enumerate(trainDL) :
    print(f'[{_}] feature {feature.shape}')
    # 로더에서 가지고 온 데이터 만큼 학습 진행

[0] feature torch.Size([5, 4])
[1] feature torch.Size([5, 4])
[2] feature torch.Size([5, 4])
[3] feature torch.Size([5, 4])
[4] feature torch.Size([5, 4])
[5] feature torch.Size([5, 4])
[6] feature torch.Size([5, 4])
[7] feature torch.Size([5, 4])
[8] feature torch.Size([5, 4])
[9] feature torch.Size([5, 4])
[10] feature torch.Size([5, 4])
[11] feature torch.Size([5, 4])
[12] feature torch.Size([5, 4])
[13] feature torch.Size([5, 4])
[14] feature torch.Size([5, 4])
[15] feature torch.Size([5, 4])
[16] feature torch.Size([5, 4])
[17] feature torch.Size([5, 4])
[18] feature torch.Size([5, 4])
[19] feature torch.Size([5, 4])
[20] feature torch.Size([5, 4])


[4] Model 클래스 정의 : 입/출력 피처 수, 층(layer) 수, 은닉층의 노드 수 <hr>
- 구조 설계
    * 입력층 : 입력 <= 피처 개수 iris 4개 
    * 은닉층 : 마음대로 알아서 
    * 출력층 : 타겟의 개수 ▶ [분류] 타겟 클래스 개수 [회귀] 1개


In [32]:
# # 로지스틱 회귀 모델 클래스 
# class Logistic(nn.Module) :
#     # 모델 구조 설정
#     def __init__(self, in_, out_) :
#         super().__init__()
#         self.layer1 = nn.Linear(in_, 4)
#         self.softmax1 = nn.Softmax()
#         self.layer2 = nn.Linear(3, out_) # setosa, vesicolor, verginica
#         self.softmax2 = nn.Softmax()
#         self.layer3 = nn.Linear()

#     def forward(self, x) :
#         y = self.layer1(x)
#         z = self.sigmoid1(y)
#         y = self.layer2(z)
#         z = self.sigmoid2(y)
#         return z




In [91]:
# # 모델 인스턴스 생성
# ll = Logistic(4,1)
# print(f'모델 구조 확인 === {ll}')

In [111]:
# 모델 클래스 정의
# 클래스명 : CMdl 
class CMdl(nn.Module) :
    
    # 구성 요소 정의
    def __init__(self, in_, out_) :
        super().__init__()
        self.input_layer = nn.Linear(in_, 100)
        self.relu=nn.ReLU()
        self.hidden_layer=nn.Linear(100, 27)
        self.output_layer = nn.Linear(27 ,out_)
    
    # 순방향 학습 진행 함수
    def forward(self, x):
        y = self.input_layer(x) # w1*1 + w2+2....Wn+b 100개 반환
        y = self.ReLU(y)            # renu() 결과 100개 반환
        y = self.hidden_layer(y)    # w1*1 + w2+2....Wn+b 27개 반환
        y = self.ReLU(y)            # 결과 3개
        y = self.output_layer(y)    # w1*1 + w2+2....Wn+b 27개 반환

        return y


[5] 학습 준비 : 실행 디바이스, 모델, 최적화, 손실함수 학습횟수, 학습함수, 평가함수, 예측 함수 <hr>


In [112]:
# 실생 디바이스 실행.
DEVICE = 'cuda'if torch.cuda.is_available() else 'cpu'

# 학습 횟수
EPOCHS = 50


In [113]:

# 모델 인스턴스
IN, OUT = my_dataset2.feature.shape[1], len(np.unique(targetSR))
mdl = CMdl(IN, OUT).to(DEVICE)
print(f'IN : {IN}, OUT : {OUT}')
print(mdl)

IN : 4, OUT : 3
CMdl(
  (input_layer): Linear(in_features=4, out_features=100, bias=True)
  (relu): ReLU()
  (hidden_layer): Linear(in_features=100, out_features=27, bias=True)
  (output_layer): Linear(in_features=27, out_features=3, bias=True)
)


In [114]:
# 손실함수
LOSS_FN = nn.CrossEntropyLoss().to(DEVICE)

# 최적화 인스턴스
import torch.optim as optim
OPTIMIZER = optim.Adam(mdl.parameters())

- 학습 및 검증 관련 함수 정의

In [115]:
# 학습 진행 함수 : 
def training() :
    # 학습 모드 => 정규화, 경사 하강법, 드랍아웃 등의 기능 활성화
    mdl.train() 

    # 배치 크기 만큼 학습 진행 및 저장
    train_loss = []
    for cnt, (feature, target) in enumerate(trainDL) :
        print(cnt, feature, target)
        feature, target = feature.to(DEVICE), target.to(DEVICE)
        target = target.squeeze
        # 학습 
        pre_target = mdl(feature)
        print(f'pre_target => {pre_target.shape}, {pre_target.ndim}D')
        print(f'target => {target.shape}, {target.ndim}D')
        
        # 손실 계산
        loss = LOSS_FN(pre_target, target)
        train_loss.append(loss)

        # W, b 업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()

        # 배치 단위 학습 메시지 출력 
        print(f'[Train {cnt} batch LOSS] ==> {loss}')

    # 학습 진행 메시지 출력
    print(f'[Train loss] ==> {loss}')

    return train_loss

In [116]:
# 검증 및 평가 진행 함수
def test() :
    pass

In [117]:
def predict():
    pass

[6] 학습 진행 <hr>

In [118]:
for eps in range(EPOCHS) :
    # 학습
    train_loss = training()
    
    # 검증
    #testing()
    print(f'[{eps}/{EPOCHS}] {sum(train_loss)/len(train_loss)}')

0 tensor([[4.4000, 3.2000, 1.3000, 0.2000],
        [5.7000, 3.0000, 4.2000, 1.2000],
        [4.8000, 3.1000, 1.6000, 0.2000],
        [5.6000, 2.9000, 3.6000, 1.3000],
        [6.9000, 3.1000, 4.9000, 1.5000]], dtype=torch.float64) tensor([[0],
        [1],
        [0],
        [1],
        [1]], dtype=torch.int32)


RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float